In [1]:
import random
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import networkx as nx

# real dataset
fb = nx.read_edgelist('data/sample-ch2017.txt')
no_node = nx.number_of_nodes(fb)
print (nx.info(fb))

Name: 
Type: Graph
Number of nodes: 31135
Number of edges: 642287
Average degree:  41.2582


In [5]:
# number of edges if the graph was complete : f(n) = (n*(n-1))/2
#cg = nx.complete_graph(no_node) # takes too long to execute thus hashed
#print(nx.number_of_nodes(cg))

fn = (no_node * (no_node - 1)) / 2
fn

484678545.0

In [2]:
# Local bridges (no common contacts)
lb = list(nx.local_bridges(fb , with_span = False))
print('Number of Local bridges in 2017', len(list(nx.local_bridges(fb, with_span = False))))

# To create a list of nodes that belong to local node. from [('a','b'), ('c','b')] to ['a','b','c','b']
new_lb = []
for tup in lb:
    for element in tup:
        new_lb.append(element)        
#nd is to remove duplicates
nd = list(set(new_lb))

Number of Local bridges in 2017 96038


In [3]:
# to find second degree contacts of a single node

def find_nodes(graph, node, distance):

     # get all nodes within distance around the query node
     nodes = set(nx.ego_graph(graph, node, radius=distance))

     # remove nodes that are not **at** the specified distance but closer: removes all the first degree contact
     if distance > 1:
      nodes -= set(nx.ego_graph(graph, node, radius=distance-1))

     return list(nodes)

# example implementation
#ba = find_nodes(fb, nd[4], 2) # reeturns a list of all the nodes tat are 2nd deg
#print((len(ba)))
#ba


In [4]:
# second degree contacts that pass through the LB of a single node
# two parameters that it takes are 1) the graph itself here fb 2) one specific local bridge (pair of nodes)
# it returns : predictive edges between the [second degree node, the node that is the part of LB]  in that same order
# returned variable is a nested list with the order 2
def seconddegree (fb, llbb):
    #llb = list(lb[0])
    b = llbb[0]
    win = []
    n = list(fb.neighbors(llbb[1]))
    pair =[]
    
    for a in range(len(n)):
        lis = []
        lis.append(n[a])
        lis.append(b)
        win.append(lis)
    return win

# example implementation
#vr = seconddegree(fb, lb[0])
#print(vr)

In [5]:
# works !! to get all the second degree edges with their respective nodes ! 
# It applies the above mentioned function to all the pair of nodes that form a local bridge
# returns nested list of order 3 . 
# non existing edges ! predicted false edges
# second node in this return set (pe) is always the node that is the part of a local bridge
def allnodesseconddegree (fb, lb):
    pe = []
    for a in range(len(lb)):
        var = seconddegree(fb, lb[a])
        pe.append(var)
    return pe

vv = allnodesseconddegree(fb, lb)
#vv

In [6]:
# This works !!
#now for the last condition ! No other mutual friend apart from ***
# returns thee nested list of all the edges(node pair) that have only one mutual friend that is a part of LB
interseclist = []
ppe = []

for a in range(len(vv)):
    for b in range(len(vv[a])):
        cc = vv[a]
        #print(type(cc))
        #print(cc[0])
        
        for c in range(len(cc[b])):
            dd = cc[b]
            #print(type(dd))
            #print(dd[0])
            #print(dd[1])
            
            if dd[0] != dd[1]:
                n1 = set(list(fb.neighbors(dd[0])))
                #print(n1)
                #print(n2)
                n2 = set(list(fb.neighbors(dd[1])))
                intersection = list(n1.intersection(n2))
                #interseclist.append(intersection)
                ls = []
                #for e in range(len(intersection)):
                
                if len(intersection) == 1 :
                    ls.append(dd[0])
                    ls.append(dd[1])
                    ppe.append(ls)
             
#print(len(interseclist))
#interseclist
#print(len((set(interseclist))))
print(len(ppe))

## Post processing clean up!! 

# remove exact replications : eg [a,b] & [a,b] then only one is kept 
b_se = set(map(tuple,ppe))  #need to convert the inner lists to tuples so they are hashable
nodup = list(map(list,b_se))
print(len(nodup))

# works : if . [a,b] and [b, a] are in the list then only one either [a,b] or [b,a] is selected
fset = set(frozenset(x) for x in nodup)
lst = [list(x) for x in fset]
print(len(lst))
lst

# 
#to remove duplications: [a,a] should be discarded
lsst = []
for a in range(len(lst)):
    ll = lst[a]
    l = []
    if ll[0] != ll[1]:
        l.append(ll[0])
        l.append(ll[1])
        lsst.append(l)
print(len(lsst))

# lsst is thhe final list to use ! lsst give a list of edges that are predicted to not get connected

7086940
3543470
3343196
3343196


In [7]:
# import 2018 dataset
fb2 = nx.read_edgelist('data/sample-ch2018.txt')
node = nx.number_of_nodes(fb2)
print (nx.info(fb2))
fn = (node * (node - 1)) / 2
print('Number off edgees that can still be established:', fn)

Name: 
Type: Graph
Number of nodes: 26276
Number of edges: 672589
Average degree:  51.1942
Number off edgees that can still be established: 345200950.0


In [10]:
edges = list(fb2.edges)
type(edges[2])

# straight nodes to match them all
lst = []
for a in range(len(lsst)):
    ls = lsst[a]
    lst.append(tuple(ls))
    

# straight list overrlap
#using sets to solve time consuming stuff
x = list(set(lst) & set(edges))
print(len(x))
#print(x)


523


In [11]:
# reversing the nodes to match them all
# this reverse command operates on the list itself :: inplace reverse operation!!

print((lsst[7])) # before reversing the list
for a in range(len(lsst)):
    lsst[a].reverse()
print((lsst[7]))

lt = []
for a in range(len(lsst)):
    ls = lsst[a]
    lt.append(tuple(ls))


# reversed list of lsst overlap
#using sets to solve time consuming stuff
y = list(set(lt) & set(edges))
print(len(y))
#print(y)


['10286548', '7140921']
['7140921', '10286548']
515


In [12]:
z = x+ y
print(len(z))
# exports
#np.savetxt("edges_that_overlap.csv", z, delimiter=",", fmt='%s')

1038


In [14]:
# to check the extra edges that was created in 2018 and did not exist in 2017
edg = list(fb.edges)
edges = list(fb2.edges)

bbaa = list(set(edg) & set(edges)) # inteersection between the two
print(len(bbaa))

# reverse checks
print(edg[7])

# to convert all the tuples to list so that they can then be reversed
l = []
for a in range(len(edg)):
    ls = edg[a]
    l.append(list(ls))

# reversing the edg list
for a in range(len(l)):
    l[a].reverse()

# reverse checks
print("Reversed list:", l[7])

#converting the list back to tuples so that the intersectio can be found
lis =[]
for a in range(len(l)):
    ls = l[a]
    lis.append(tuple(ls))
    
print("Reversed list:",  lis[7])    
   
bbbaa = list(set(lis).intersection(set(edges)))
print(len(bbbaa))

#intesecction of 2017 and 2018 
is78 = bbbaa + bbaa
print('Inteersection length is:', len(is78))

332631
('7394358', '7845466')
Reversed list: ['7845466', '7394358']
Reversed list: ('7845466', '7394358')
290004
Inteersection length is: 622635


In [15]:
# difference
diff = list((set(edges)).difference(set(is78)))
print(len(diff))


49954
